# Credit Risk Analysis

### For Google Colab

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !cp drive/MyDrive/Credit_Risk_Analysis/* .

In [3]:
# !pip install catboost

## Step-1: Import Libraries

### Import the necessary base libraries

In [4]:
# Base libraries
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import os.path
import pickle

### Import visualization libraries

In [5]:
# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

### Import other libraries

In [6]:
# Othe libraries
from sklearn import metrics
from sklearn.model_selection import train_test_split
# from sklearn.impute import SimpleImputer
from catboost import CatBoostClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score

### Import helper functions

In [7]:
# import helper functions
from Helper_Module_Credit_Risk_Analysis import *
Custom_Helper_Module()


    Available General Custom Functions: 

    Check_Missing_Values(input_dataset)
    Check_Feature_Details(input_dataset, input_feature)
    Create_Dummy_Variables(input_dataset, input_feature_list)
    Preliminary_Feature_Selection(input_X_train, input_y_train)
    Check_Correlation(input_X_train)
    Check_Multicollinearity(input_X_train, numerical_feature_list)
    Make_Feature_Selection(input_X_train, input_y_train, max_validation_round)
    Remove_Outlies(input_dataset, input_features)

    Convert_Datetime_To_Months(df, column)
    Convert_Loan_Tenure_To_Months(df, column)
    Convert_Employment_Length_To_Years(df, column)
    


## Step-2: Import Dataset

In [8]:
dataset = pd.read_csv('Raw_Dataset/Loan_Dataset.csv')
# dataset = pd.read_csv('drive/MyDrive/Dataset/Credit_Risk_Analysis/Loan_Dataset.csv')

In [9]:
# copy the dataset into a new dataframe for further processing
imported_dataset = dataset.copy()

## Step-3: Data Exploration

In [10]:
# display all columns
pd.options.display.max_columns = None

In [11]:
# shape of the dataset
imported_dataset.shape

(466285, 72)

In [12]:
# fast look of the data set
imported_dataset.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-11,Fully Paid,n,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-85,1.0,NaN,NaN,3.0,0.0,13648,83.7,9.0,f,0.0,0.0,5861.071414,5831.78,5000.00,861.07,0.00,0.00,0.00,Jan-15,171.62,NaN,Jan-16,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-11,Charged Off,n,car,bike,309xx,GA,1.00,0.0,Apr-99,5.0,NaN,NaN,3.0,0.0,1687,9.4,4.0,f,0.0,0.0,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-13,119.66,NaN,Sep-13,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-11,Fully Paid,n,small_business,real estate business,606xx,IL,8.72,0.0,Nov-01,2.0,NaN,NaN,2.0,0.0,2956,98.5,10.0,f,0.0,0.0,3003.653644,3003.65,2400.00,603.65,0.00,0.00,0.00,Jun-14,649.91,NaN,Jan-16,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-11,Fully Paid,n,other,personel,917xx,CA,20.00,0.0,Feb-96,1.0,35.0,NaN,10.0,0.0,5598,21.0,37.0,f,0.0,0.0,12226.302210,12226.30,10000.00,2209.33,16.97,0.00,0.00,Jan-15,357.48,NaN,Jan-15,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-11,Current,n,other,Personal,972xx,OR,17.94,0.0,Jan-96,0.0,38.0,NaN,15.0,0.0,27783,53.9,38.0,f,766.9,766.9,3242.170000,3242.17,2233.10,1009.07,0.00,0.00,0.00,Jan-16,67.79,Feb-16,Jan-16,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
imported_dataset.describe(include='all')

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
count,4.662850e+05,4.662850e+05,466285.000000,466285.000000,466285.000000,466285,466285.000000,466285.000000,466285,466285,438697,445277,466285,4.662810e+05,466285,466285,466285,466285,466285,466265,466285,466285,466285.000000,466256.000000,466256,466256.000000,215934.000000,62638.000000,466256.000000,466256.000000,4.662850e+05,465945.000000,466256.000000,466285,466285.000000,466285.000000,466285.000000,466285.000000,466285.000000,466285.000000,466285.000000,466285.000000,466285.000000,465909,466285.000000,239071,466243,466140.000000,98974.000000,466285.0,466285,0.0,0.0,0.0,466256.000000,3.960090e+05,3.960090e+05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.960090e+05,0.0,0.0,0.0
unique,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,7,35,205475,11,6,NaN,3,91,9,2,14,63098,888,50,NaN,NaN,664,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98,NaN,100,103,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,36 months,NaN,NaN,B,B3,Teacher,10+ years,MORTGAGE,NaN,Verified,Oct-14,Current,n,debt_consolidation,Debt consolidation,945xx,CA,NaN,NaN,Oct-00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jan-16,NaN,Feb-16,Jan-16,NaN,NaN,NaN,INDIVIDUAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,337953,NaN,NaN,136929,31686,5399,150049,235875,NaN,168055,38782,224226,466276,274195,164075,5304,71450,NaN,NaN,3674,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,303005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,179620,NaN,208393,327699,NaN,NaN,NaN,466285,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,1.307973e+07,1.459766e+07,14317.277577,14291.801044,14222.329888,NaN,13.829236,432.061201,NaN,NaN,NaN,NaN,NaN,7.327738e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.218758,0.284678,NaN,0.804745,34.104430,74.306012,11.187069,0.160564,1.623020e+04,56.176947,25.064430,NaN,4410.062342,4408.452258,11540.686220,11469.892747,8866.014657,2588.677225,0.650129,85.344211,8.961534,NaN,3123.913796,NaN,NaN,0.009085,42.852547,1.0,NaN,NaN,NaN,NaN,0.004002,1.919135e+02,1.388017e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.037909e+04,NaN,NaN,NaN
std,1.089371e+07,1.168237e+07,8286.509164,8274.371300,8297.637788,NaN,4.357587,243.485550,NaN,NaN,NaN,NaN,NaN,5.496357e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.851121,0.797365,NaN,1.091598,21.778487,30.357653,4.987526,0.510863,2.067625e+04,23.732628,11.600141,NaN,6355.078769,6353.198001,8265.627112,8254.157579,7031.687997,2483.809661,5.265730,552.216084,85.491437,NaN,5554.737393,NaN,NaN,0.108648,21.662591,0.0,NaN,NaN,NaN,NaN,0.068637,1.463021e+04,1.521147e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.724713e+04,NaN,NaN,NaN
min,5.473400e+04,7.047300e+04,500.000000,500.000000,0.000000,NaN,5.420000,15.670000,NaN,NaN,NaN,NaN,NaN,1.896000e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,1.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000

In [14]:
Check_Missing_Values(imported_dataset)

Following featues have more than 80% missing values:  18
['mths_since_last_record', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'open_acc_6m', 'open_il_6m', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'inq_fi', 'total_cu_tl', 'inq_last_12m']


,Missing_Data,Missing_Data (%),Data_Type
id,0,0.00,int64
member_id,0,0.00,int64
loan_amnt,0,0.00,int64
funded_amnt,0,0.00,int64
funded_amnt_inv,0,0.00,float64
...,...,...,...
all_util,466285,100.00,float64
total_rev_hi_lim,70276,15.07,float64
inq_fi,466285,100.00,float64
total_cu_tl,466285,100.00,float64


In [15]:
# shape of the datset
imported_dataset.shape

(466285, 72)

In [16]:
categorical_variable_list = list(imported_dataset.select_dtypes(include=['object']).columns)
print(len(categorical_variable_list))
print(categorical_variable_list)

20
['term', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code', 'addr_state', 'earliest_cr_line', 'initial_list_status', 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d', 'application_type']


In [17]:
imported_dataset[categorical_variable_list].head()

,term,grade,sub_grade,emp_title,emp_length,home_ownership,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,earliest_cr_line,initial_list_status,last_pymnt_d,next_pymnt_d,last_credit_pull_d,application_type
0,36 months,B,B2,NaN,10+ years,RENT,Verified,Dec-11,Fully Paid,n,credit_card,Computer,860xx,AZ,Jan-85,f,Jan-15,NaN,Jan-16,INDIVIDUAL
1,60 months,C,C4,Ryder,< 1 year,RENT,Source Verified,Dec-11,Charged Off,n,car,bike,309xx,GA,Apr-99,f,Apr-13,NaN,Sep-13,INDIVIDUAL
2,36 months,C,C5,NaN,10+ years,RENT,Not Verified,Dec-11,Fully Paid,n,small_business,real estate business,606xx,IL,Nov-01,f,Jun-14,NaN,Jan-16,INDIVIDUAL
3,36 months,C,C1,AIR RESOURCES BOARD,10+ years,RENT,Source Verified,Dec-11,Fully Paid,n,other,personel,917xx,CA,Feb-96,f,Jan-15,NaN,Jan-15,INDIVIDUAL
4,60 months,B,B5,University Medical Group,1 year,RENT,Source Verified,Dec-11,Current,n,other,Personal,972xx,OR,Jan-96,f,Jan-16,Feb-16,Jan-16,INDIVIDUAL


In [18]:
# term can be converted to months
# emp_length can be converted to years
# issue_d, earliest_cr_line, last_pymnt_d, next_pymnt_d, last_credit_pull_d are in datetime format 
# and can be converted to months
# loan_status is our target variable and need some formatting

In [19]:
numerical_variable_list = list(imported_dataset.select_dtypes(include=['number']).columns)
print(len(numerical_variable_list))
print(numerical_variable_list)

52
['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'int_rate', 'installment', 'annual_inc', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_amnt', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'policy_code', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_il_6m', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m']


In [20]:
Check_Missing_Values(imported_dataset)

Following featues have more than 80% missing values:  18
['mths_since_last_record', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'open_acc_6m', 'open_il_6m', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'inq_fi', 'total_cu_tl', 'inq_last_12m']


,Missing_Data,Missing_Data (%),Data_Type
id,0,0.00,int64
member_id,0,0.00,int64
loan_amnt,0,0.00,int64
funded_amnt,0,0.00,int64
funded_amnt_inv,0,0.00,float64
...,...,...,...
all_util,466285,100.00,float64
total_rev_hi_lim,70276,15.07,float64
inq_fi,466285,100.00,float64
total_cu_tl,466285,100.00,float64


In [21]:
# there are 18 features which have more than 80% missing values
# any technique to impute those missing values is most likely to indroduce errors
# we will simply drop them
high_missing_value_columns = ['mths_since_last_record', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'open_acc_6m', 'open_il_6m', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'inq_fi', 'total_cu_tl', 'inq_last_12m']

In [22]:
imported_dataset.drop(high_missing_value_columns, axis = 1, inplace=True)

In [23]:
# certain features like id, member_id are not related to credit risk
# some features like recoveries, collection_recovery_fee are applicable only after default
# since our purpose is to predit the probability of default, we can drop those features
other_columns_to_drop = ['id', 'member_id', 'recoveries', 'collection_recovery_fee']

In [24]:
imported_dataset.drop(other_columns_to_drop, axis = 1, inplace=True)

In [25]:
imported_dataset.shape

(466285, 50)

## Step-4: Format Dataset

### Format the target variable

In [26]:
# since we want to predict probability of deafult, 'loan_status' is our target varible
# explore the unique values in 'loan_status' column
imported_dataset['loan_status'].value_counts(normalize = True)

Current                                                0.480878
Fully Paid                                             0.396193
Charged Off                                            0.091092
Late (31-120 days)                                     0.014798
In Grace Period                                        0.006747
Does not meet the credit policy. Status:Fully Paid     0.004263
Late (16-30 days)                                      0.002612
Default                                                0.001784
Does not meet the credit policy. Status:Charged Off    0.001632
Name: loan_status, dtype: float64

In [27]:
# create a new column based on the loan_status column that will be our target variable
imported_dataset['good_loan'] = np.where(imported_dataset.loc[:, 'loan_status'].isin(['Charged Off', 
                                                                                            'Default', 
                                                                                            'Late (31-120 days)', 
                                                                                            'Does not meet the credit policy. Status:Charged Off']), 0, 1)

In [28]:
# drop the original 'loan_status' column
imported_dataset.drop(columns = ['loan_status'], inplace = True)

### Convert datatime columns to months

In [29]:
# the folloing columns have datetime format
# 'issue_d', 'earliest_cr_line', 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d'
# convert datetime columns to datetime format and 
# create a new column as a difference between today 
# and the respective date with prefix months_since_
# also drop the original column

Convert_Datetime_To_Months(imported_dataset, 'issue_d')
Convert_Datetime_To_Months(imported_dataset, 'earliest_cr_line')
Convert_Datetime_To_Months(imported_dataset, 'last_pymnt_d')
Convert_Datetime_To_Months(imported_dataset, 'next_pymnt_d')
Convert_Datetime_To_Months(imported_dataset, 'last_credit_pull_d')

### Format some other columns

In [30]:
# convert loan tenure to months
Convert_Loan_Tenure_To_Months(imported_dataset, 'term')

In [31]:
# convert employment length to years
Convert_Employment_Length_To_Years(imported_dataset, 'emp_length')

In [32]:
formatted_dataset = imported_dataset.copy()

In [33]:
formatted_dataset.shape

(466285, 50)

## Step-5: Train Test Split

In [34]:
# decoupling the dependent and independent variables
X = formatted_dataset.drop('good_loan', axis = 1)
y = formatted_dataset['good_loan']

In [35]:
# Train, Test split
# from now on till the model training we will only use X_train, y_train
# X_test and y_test will only be used during model testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)

In [36]:
X_train.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,last_pymnt_amnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,months_since_issue_d,months_since_earliest_cr_line,months_since_last_pymnt_d,months_since_next_pymnt_d,months_since_last_credit_pull_d
456615,15000,15000,15000.0,36,8.90,476.30,A,A5,RN Team Lead,7.0,MORTGAGE,80000.0,Source Verified,n,credit_card,Credit card refinancing,541xx,WI,17.01,1.0,0.0,20.0,19.0,0.0,20699,59.0,32.0,w,0.00,0.00,16561.02,16561.02,15000.00,1561.02,0.0,8940.22,0.0,NaN,1,INDIVIDUAL,0.0,0.0,143586.0,35100.0,84.0,301.0,67.0,NaN,60.0
451541,8000,8000,8000.0,60,18.25,204.24,D,D3,legal assistant,10.0,OWN,44000.0,Verified,n,other,Other,370xx,TN,23.46,0.0,1.0,NaN,12.0,0.0,13245,32.1,25.0,f,5746.54,5746.54,4697.52,4697.52,2253.46,2444.06,0.0,204.24,0.0,NaN,1,INDIVIDUAL,0.0,0.0,180443.0,41300.0,84.0,306.0,60.0,60.0,60.0
394474,12150,12150,12100.0,60,18.92,314.65,D,D4,administrative assistant,3.0,OWN,27000.0,Source Verified,n,credit_card,Credit card refinancing,376xx,TN,31.07,1.0,0.0,10.0,9.0,0.0,7172,73.2,22.0,f,9282.24,9244.04,6293.00,6267.10,2867.76,3425.24,0.0,314.65,0.0,NaN,1,INDIVIDUAL,0.0,0.0,34197.0,9800.0,80.0,364.0,60.0,59.0,60.0
110294,10000,10000,10000.0,36,6.03,304.36,A,A1,Bristol Metals,3.0,MORTGAGE,33000.0,Not Verified,n,debt_consolidation,loan consolidation,376xx,TN,9.16,0.0,0.0,NaN,5.0,0.0,2138,21.2,17.0,w,0.00,0.00,10858.01,10858.01,10000.00,858.01,0.0,3553.37,0.0,NaN,1,INDIVIDUAL,0.0,0.0,77959.0,10100.0,89.0,229.0,64.0,NaN,64.0
139343,15825,15825,15825.0,36,12.12,526.53,B,B3,waste connections,10.0,MORTGAGE,59000.0,Verified,n,debt_consolidation,3 year consoladtion,986xx,WA,17.94,0.0,0.0,NaN,7.0,0.0,30326,93.6,31.0,f,0.00,0.00,4736.52,4736.52,3435.17,1301.35,0.0,526.53,0.0,NaN,1,INDIVIDUAL,0.0,0.0,187370.0,32400.0,92.0,499.0,83.0,NaN,60.0


In [37]:
print(X_train.shape)
print(X_test.shape)

(373028, 49)
(93257, 49)


In [38]:
categorical_feature_list = list(X_train.select_dtypes(include=['object']).columns)
print(len(categorical_feature_list))
print(categorical_feature_list)

12
['grade', 'sub_grade', 'emp_title', 'home_ownership', 'verification_status', 'pymnt_plan', 'purpose', 'title', 'zip_code', 'addr_state', 'initial_list_status', 'application_type']


In [39]:
numerical_feature_list = list(X_train.select_dtypes(include=['number']).columns)
print(len(numerical_feature_list))
print(numerical_feature_list)

37
['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'emp_length', 'annual_inc', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'mths_since_last_delinq', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'last_pymnt_amnt', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'policy_code', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim', 'months_since_issue_d', 'months_since_earliest_cr_line', 'months_since_last_pymnt_d', 'months_since_next_pymnt_d', 'months_since_last_credit_pull_d']


In [40]:
# backup copy the dataset for feature reference
backup_X_train = X_train.copy()
backup_X_test = X_test.copy()

## Step-6: Handling Missing Values

In [41]:
Check_Missing_Values(X_train)

Following featues have more than 80% missing values:  0
[]


,Missing_Data,Missing_Data (%),Data_Type
loan_amnt,0,0.00,int64
funded_amnt,0,0.00,int64
funded_amnt_inv,0,0.00,float64
term,0,0.00,int64
int_rate,0,0.00,float64
installment,0,0.00,float64
grade,0,0.00,object
sub_grade,0,0.00,object
emp_title,22047,5.91,object
emp_length,16793,4.50,float64


In [42]:
X_train.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,last_pymnt_amnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,months_since_issue_d,months_since_earliest_cr_line,months_since_last_pymnt_d,months_since_next_pymnt_d,months_since_last_credit_pull_d
456615,15000,15000,15000.0,36,8.90,476.30,A,A5,RN Team Lead,7.0,MORTGAGE,80000.0,Source Verified,n,credit_card,Credit card refinancing,541xx,WI,17.01,1.0,0.0,20.0,19.0,0.0,20699,59.0,32.0,w,0.00,0.00,16561.02,16561.02,15000.00,1561.02,0.0,8940.22,0.0,NaN,1,INDIVIDUAL,0.0,0.0,143586.0,35100.0,84.0,301.0,67.0,NaN,60.0
451541,8000,8000,8000.0,60,18.25,204.24,D,D3,legal assistant,10.0,OWN,44000.0,Verified,n,other,Other,370xx,TN,23.46,0.0,1.0,NaN,12.0,0.0,13245,32.1,25.0,f,5746.54,5746.54,4697.52,4697.52,2253.46,2444.06,0.0,204.24,0.0,NaN,1,INDIVIDUAL,0.0,0.0,180443.0,41300.0,84.0,306.0,60.0,60.0,60.0
394474,12150,12150,12100.0,60,18.92,314.65,D,D4,administrative assistant,3.0,OWN,27000.0,Source Verified,n,credit_card,Credit card refinancing,376xx,TN,31.07,1.0,0.0,10.0,9.0,0.0,7172,73.2,22.0,f,9282.24,9244.04,6293.00,6267.10,2867.76,3425.24,0.0,314.65,0.0,NaN,1,INDIVIDUAL,0.0,0.0,34197.0,9800.0,80.0,364.0,60.0,59.0,60.0
110294,10000,10000,10000.0,36,6.03,304.36,A,A1,Bristol Metals,3.0,MORTGAGE,33000.0,Not Verified,n,debt_consolidation,loan consolidation,376xx,TN,9.16,0.0,0.0,NaN,5.0,0.0,2138,21.2,17.0,w,0.00,0.00,10858.01,10858.01,10000.00,858.01,0.0,3553.37,0.0,NaN,1,INDIVIDUAL,0.0,0.0,77959.0,10100.0,89.0,229.0,64.0,NaN,64.0
139343,15825,15825,15825.0,36,12.12,526.53,B,B3,waste connections,10.0,MORTGAGE,59000.0,Verified,n,debt_consolidation,3 year consoladtion,986xx,WA,17.94,0.0,0.0,NaN,7.0,0.0,30326,93.6,31.0,f,0.00,0.00,4736.52,4736.52,3435.17,1301.35,0.0,526.53,0.0,NaN,1,INDIVIDUAL,0.0,0.0,187370.0,32400.0,92.0,499.0,83.0,NaN,60.0


In [43]:
Replace_Missing_Categorical_Values(X_train, categorical_feature_list)

In [44]:
Replace_Missing_Numerical_Values(X_train, numerical_feature_list)

In [45]:
X_train.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,last_pymnt_amnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,months_since_issue_d,months_since_earliest_cr_line,months_since_last_pymnt_d,months_since_next_pymnt_d,months_since_last_credit_pull_d
456615,15000.0,15000.0,15000.0,36.0,8.90,476.30,A,A5,RN Team Lead,7.0,MORTGAGE,80000.0,Source Verified,n,credit_card,Credit card refinancing,541xx,WI,17.01,1.0,0.0,20.0,19.0,0.0,20699.0,59.0,32.0,w,0.00,0.00,16561.02,16561.02,15000.00,1561.02,0.0,8940.22,0.0,42.0,1.0,INDIVIDUAL,0.0,0.0,143586.0,35100.0,84.0,301.0,67.0,59.0,60.0
451541,8000.0,8000.0,8000.0,60.0,18.25,204.24,D,D3,legal assistant,10.0,OWN,44000.0,Verified,n,other,Other,370xx,TN,23.46,0.0,1.0,31.0,12.0,0.0,13245.0,32.1,25.0,f,5746.54,5746.54,4697.52,4697.52,2253.46,2444.06,0.0,204.24,0.0,42.0,1.0,INDIVIDUAL,0.0,0.0,180443.0,41300.0,84.0,306.0,60.0,60.0,60.0
394474,12150.0,12150.0,12100.0,60.0,18.92,314.65,D,D4,administrative assistant,3.0,OWN,27000.0,Source Verified,n,credit_card,Credit card refinancing,376xx,TN,31.07,1.0,0.0,10.0,9.0,0.0,7172.0,73.2,22.0,f,9282.24,9244.04,6293.00,6267.10,2867.76,3425.24,0.0,314.65,0.0,42.0,1.0,INDIVIDUAL,0.0,0.0,34197.0,9800.0,80.0,364.0,60.0,59.0,60.0
110294,10000.0,10000.0,10000.0,36.0,6.03,304.36,A,A1,Bristol Metals,3.0,MORTGAGE,33000.0,Not Verified,n,debt_consolidation,loan consolidation,376xx,TN,9.16,0.0,0.0,31.0,5.0,0.0,2138.0,21.2,17.0,w,0.00,0.00,10858.01,10858.01,10000.00,858.01,0.0,3553.37,0.0,42.0,1.0,INDIVIDUAL,0.0,0.0,77959.0,10100.0,89.0,229.0,64.0,59.0,64.0
139343,15825.0,15825.0,15825.0,36.0,12.12,526.53,B,B3,waste connections,10.0,MORTGAGE,59000.0,Verified,n,debt_consolidation,3 year consoladtion,986xx,WA,17.94,0.0,0.0,31.0,7.0,0.0,30326.0,93.6,31.0,f,0.00,0.00,4736.52,4736.52,3435.17,1301.35,0.0,526.53,0.0,42.0,1.0,INDIVIDUAL,0.0,0.0,187370.0,32400.0,92.0,499.0,83.0,59.0,60.0


In [46]:
Check_Missing_Values(X_train)

Following featues have more than 80% missing values:  0
[]


,Missing_Data,Missing_Data (%),Data_Type
loan_amnt,0,0.0,float64
funded_amnt,0,0.0,float64
funded_amnt_inv,0,0.0,float64
term,0,0.0,float64
int_rate,0,0.0,float64
installment,0,0.0,float64
grade,0,0.0,object
sub_grade,0,0.0,object
emp_title,0,0.0,object
emp_length,0,0.0,float64


In [47]:
X_train.shape

(373028, 49)

## Step-5: Get Dummies

In [48]:
X_train.shape

(373028, 49)

In [49]:
X_train.describe(include='all')

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,last_pymnt_amnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,months_since_issue_d,months_since_earliest_cr_line,months_since_last_pymnt_d,months_since_next_pymnt_d,months_since_last_credit_pull_d
count,373028.000000,373028.000000,373028.000000,373028.000000,373028.000000,373028.000000,373028,373028,373028,373028.000000,373028,3.730280e+05,373028,373028,373028,373028,373028,373028,373028.000000,373028.000000,373028.000000,373028.000000,373028.000000,373028.000000,3.730280e+05,373028.000000,373028.000000,373028,373028.000000,373028.000000,373028.000000,373028.000000,373028.000000,373028.000000,373028.000000,373028.000000,373028.000000,373028.000000,373028.0,373028,373028.000000,3.730280e+05,3.730280e+05,3.730280e+05,373028.000000,373028.000000,373028.000000,373028.000000,373028.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,7,35,170225,NaN,5,NaN,3,2,14,52169,877,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,B,B3,missing_value_emp_title,NaN,MORTGAGE,NaN,Verified,n,debt_consolidation,Debt consolidation,945xx,CA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INDIVIDUAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,109344,25207,22047,NaN,188739,NaN,134571,373021,219196,131255,4211,56992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,242325,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,373028,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,14317.583667,14292.268945,14223.416536,42.615847,13.833770,431.986953,NaN,NaN,NaN,5.994421,NaN,7.334178e+04,NaN,NaN,NaN,NaN,NaN,NaN,17.222219,0.284121,0.805234,32.441021,11.192104,0.160280,1.625483e+04,56.193325,25.069325,NaN,4409.242313,4407.639480,11540.311620,11470.075962,8863.673804,2589.754029,0.650353,3124.798466,0.009053,42.179649,1.0,NaN,0.003943,1.710375e+02,1.304728e+05,2.925643e+04,88.252485,279.292450,68.283912,59.380274,64.041485
std,8291.196247,8279.296230,8302.401020,10.724327,4.354663,243.518563,NaN,NaN,NaN,3.542932,NaN,5.613212e+04,NaN,NaN,NaN,NaN,NaN,NaN,7.850161,0.794508,1.093425,14.899779,4.990453,0.493881,2.096174e+04,23.720280,11.591456,NaN,6357.128699,6355.251836,8272.661827,8260.698017,7036.196421,2487.839259,5.280674,5559.815170,0.110350,9.989807,0.0,NaN,0.068125,1.506198e+04,1.422551e+05,3.596587e+04,14.339074,86.369963,12.800250,4.383706,9.630568
min,500.000000,500.000000,0.000000,36.000000,5.420000,15.670000,NaN,NaN,NaN,0.000000,NaN,1.896000e+03,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,1.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,NaN,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,73.000000,110.000000,60.000000,58.000000,60.000000
25%,8000.000000,8000.000000,8000.000000,36.000000,10.990000,256.670000,NaN,NaN,NaN,3.000000,NaN,4.500000e+04,NaN,NaN,NaN,NaN,NaN,NaN,11.370000,0.000000,0.000000,31.000000,8.000000,0.000000,6.416000e+03,39.300000,17.000000,NaN,0.000000,0.000000,5550.586971,5497.460000,3705.630000,956.600000,0.000000,312.387500,0.000000,42.000000,1.0,NaN,0.000000,0.000000e+00,3.365975e+04,1.500000e+04,78.000000,221.000000,60.000000,59.000000,60.000000
50%,12000.000000,12000.000000,12000.000000,36.000000,13.660000,379.760000,NaN,NaN,NaN,6.000000,NaN,6.300000e+04,NaN,NaN,NaN,NaN,NaN,NaN,16.870000,0.000000,0.000000,31.000000,10.000000,0.000000,1.176550e+04,57.600000,23.000000,NaN,443.970000,443.800000,9419.576549,9355

In [50]:
# now there are some categorical features, where there are lot of categories
# we will take top 10 categories for each feature and replace the remaining categories by 'Other'

In [51]:
categorical_feature_list = list(X_train.select_dtypes(include=['object']).columns)
print(len(categorical_feature_list))
print(categorical_feature_list)

12
['grade', 'sub_grade', 'emp_title', 'home_ownership', 'verification_status', 'pymnt_plan', 'purpose', 'title', 'zip_code', 'addr_state', 'initial_list_status', 'application_type']


In [52]:
Reduce_Category(X_train, categorical_feature_list)

Reducing categories up to 10 (top) categories for each feature ...



In [53]:
X_train.describe(include='all')

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,last_pymnt_amnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,months_since_issue_d,months_since_earliest_cr_line,months_since_last_pymnt_d,months_since_next_pymnt_d,months_since_last_credit_pull_d
count,373028.000000,373028.000000,373028.000000,373028.000000,373028.000000,373028.000000,373028,373028,373028,373028.000000,373028,3.730280e+05,373028,373028,373028,373028,373028,373028,373028.000000,373028.000000,373028.000000,373028.000000,373028.000000,373028.000000,3.730280e+05,373028.000000,373028.000000,373028,373028.000000,373028.000000,373028.000000,373028.000000,373028.000000,373028.000000,373028.000000,373028.000000,373028.000000,373028.000000,373028.0,373028,373028.000000,3.730280e+05,3.730280e+05,3.730280e+05,373028.000000,373028.000000,373028.000000,373028.000000,373028.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,7,10,10,NaN,5,NaN,3,2,10,9,10,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,B,Other,Other,NaN,MORTGAGE,NaN,Verified,n,debt_consolidation,Other,Other,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INDIVIDUAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,109344,181113,334038,NaN,188739,NaN,134571,373021,219196,152546,340466,162888,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,242325,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,373028,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,14317.583667,14292.268945,14223.416536,42.615847,13.833770,431.986953,NaN,NaN,NaN,5.994421,NaN,7.334178e+04,NaN,NaN,NaN,NaN,NaN,NaN,17.222219,0.284121,0.805234,32.441021,11.192104,0.160280,1.625483e+04,56.193325,25.069325,NaN,4409.242313,4407.639480,11540.311620,11470.075962,8863.673804,2589.754029,0.650353,3124.798466,0.009053,42.179649,1.0,NaN,0.003943,1.710375e+02,1.304728e+05,2.925643e+04,88.252485,279.292450,68.283912,59.380274,64.041485
std,8291.196247,8279.296230,8302.401020,10.724327,4.354663,243.518563,NaN,NaN,NaN,3.542932,NaN,5.613212e+04,NaN,NaN,NaN,NaN,NaN,NaN,7.850161,0.794508,1.093425,14.899779,4.990453,0.493881,2.096174e+04,23.720280,11.591456,NaN,6357.128699,6355.251836,8272.661827,8260.698017,7036.196421,2487.839259,5.280674,5559.815170,0.110350,9.989807,0.0,NaN,0.068125,1.506198e+04,1.422551e+05,3.596587e+04,14.339074,86.369963,12.800250,4.383706,9.630568
min,500.000000,500.000000,0.000000,36.000000,5.420000,15.670000,NaN,NaN,NaN,0.000000,NaN,1.896000e+03,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,1.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,NaN,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,73.000000,110.000000,60.000000,58.000000,60.000000
25%,8000.000000,8000.000000,8000.000000,36.000000,10.990000,256.670000,NaN,NaN,NaN,3.000000,NaN,4.500000e+04,NaN,NaN,NaN,NaN,NaN,NaN,11.370000,0.000000,0.000000,31.000000,8.000000,0.000000,6.416000e+03,39.300000,17.000000,NaN,0.000000,0.000000,5550.586971,5497.460000,3705.630000,956.600000,0.000000,312.387500,0.000000,42.000000,1.0,NaN,0.000000,0.000000e+00,3.365975e+04,1.500000e+04,78.000000,221.000000,60.000000,59.000000,60.000000
50%,12000.000000,12000.000000,12000.000000,36.000000,13.660000,379.760000,NaN,NaN,NaN,6.000000,NaN,6.300000e+04,NaN,NaN,NaN,NaN,NaN,NaN,16.870000,0.000000,0.000000,31.000000,10.000000,0.000000,1.176550e+04,57.600000,23.000000,NaN,443.970000,443.800000,9419.576549,9355.460000,6803.485000,1818.4250

In [54]:
# now let's get the dummies
X_train = pd.get_dummies(X_train, drop_first=True)

In [55]:
X_train.shape

(373028, 104)

## Step-8: Preliminary Feature Selection

In [56]:
# run time ~ 5 min
if not os.path.exists('preliminary_feature_list.txt'):
    Preliminary_Feature_Selection(X_train, y_train)

In [57]:
with open('preliminary_feature_list.txt') as f:
    preliminary_feature_list = f.read().splitlines()

In [58]:
preliminary_feature_list_X_train = X_train[preliminary_feature_list].copy()
X_train = preliminary_feature_list_X_train.copy()

In [59]:
X_train.columns

Index(['total_rec_prncp', 'months_since_last_pymnt_d', 'last_pymnt_amnt',
       'total_pymnt', 'total_pymnt_inv', 'funded_amnt', 'out_prncp',
       'loan_amnt', 'funded_amnt_inv', 'installment', 'out_prncp_inv',
       'total_rec_int', 'months_since_issue_d', 'total_rec_late_fee',
       'int_rate', 'months_since_last_credit_pull_d', 'term', 'tot_cur_bal',
       'revol_bal', 'dti', 'total_rev_hi_lim', 'revol_util', 'annual_inc',
       'months_since_earliest_cr_line', 'total_acc', 'open_acc',
       'mths_since_last_delinq', 'grade_E', 'emp_length', 'title_Other',
       'inq_last_6mths', 'mths_since_last_major_derog',
       'title_Debt consolidation', 'initial_list_status_w', 'grade_F',
       'months_since_next_pymnt_d', 'verification_status_Verified', 'grade_D'],
      dtype='object')

In [60]:
X_train.shape

(373028, 38)

In [61]:
X_train.head()

,total_rec_prncp,months_since_last_pymnt_d,last_pymnt_amnt,total_pymnt,total_pymnt_inv,funded_amnt,out_prncp,loan_amnt,funded_amnt_inv,installment,out_prncp_inv,total_rec_int,months_since_issue_d,total_rec_late_fee,int_rate,months_since_last_credit_pull_d,term,tot_cur_bal,revol_bal,dti,total_rev_hi_lim,revol_util,annual_inc,months_since_earliest_cr_line,total_acc,open_acc,mths_since_last_delinq,grade_E,emp_length,title_Other,inq_last_6mths,mths_since_last_major_derog,title_Debt consolidation,initial_list_status_w,grade_F,months_since_next_pymnt_d,verification_status_Verified,grade_D
456615,15000.00,67.0,8940.22,16561.02,16561.02,15000.0,0.00,15000.0,15000.0,476.30,0.00,1561.02,84.0,0.0,8.90,60.0,36.0,143586.0,20699.0,17.01,35100.0,59.0,80000.0,301.0,32.0,19.0,20.0,0,7.0,0,0.0,42.0,0,1,0,59.0,0,0
451541,2253.46,60.0,204.24,4697.52,4697.52,8000.0,5746.54,8000.0,8000.0,204.24,5746.54,2444.06,84.0,0.0,18.25,60.0,60.0,180443.0,13245.0,23.46,41300.0,32.1,44000.0,306.0,25.0,12.0,31.0,0,10.0,1,1.0,42.0,0,0,0,60.0,1,1
394474,2867.76,60.0,314.65,6293.00,6267.10,12150.0,9282.24,12150.0,12100.0,314.65,9244.04,3425.24,80.0,0.0,18.92,60.0,60.0,34197.0,7172.0,31.07,9800.0,73.2,27000.0,364.0,22.0,9.0,10.0,0,3.0,0,0.0,42.0,0,0,0,59.0,0,1
110294,10000.00,64.0,3553.37,10858.01,10858.01,10000.0,0.00,10000.0,10000.0,304.36,0.00,858.01,89.0,0.0,6.03,64.0,36.0,77959.0,2138.0,9.16,10100.0,21.2,33000.0,229.0,17.0,5.0,31.0,0,3.0,1,0.0,42.0,0,1,0,59.0,0,0
139343,3435.17,83.0,526.53,4736.52,4736.52,15825.0,0.00,15825.0,15825.0,526.53,0.00,1301.35,92.0,0.0,12.12,60.0,36.0,187370.0,30326.0,17.94,32400.0,93.6,59000.0,499.0,31.0,7.0,31.0,0,10.0,1,0.0,42.0,0,0,0,59.0,1,0


## Step-9: Check Correlation and Multicollinearity

### Get correlation qualified training dataset

In [62]:
X_train.shape

(373028, 38)

In [63]:
if not os.path.exists('correlation_qualified_feature_list.txt'):
    Check_Correlation(X_train)

In [64]:
with open('correlation_qualified_feature_list.txt') as f:
    correlation_qualified_feature_list = f.read().splitlines()

In [65]:
correlation_qualified_X_train = X_train[correlation_qualified_feature_list].copy()

In [66]:
X_train = correlation_qualified_X_train.copy()
X_train.shape

(373028, 32)

### Get multicollinearity qualified training dataset

In [67]:
X_train.shape

(373028, 32)

In [68]:
numerical_feature_list = list(X_train.select_dtypes(include=['number']).columns)
print(len(numerical_feature_list))
print(numerical_feature_list)

32
['total_rec_prncp', 'months_since_last_pymnt_d', 'last_pymnt_amnt', 'funded_amnt', 'out_prncp', 'total_rec_int', 'months_since_issue_d', 'total_rec_late_fee', 'int_rate', 'months_since_last_credit_pull_d', 'term', 'tot_cur_bal', 'revol_bal', 'dti', 'total_rev_hi_lim', 'revol_util', 'annual_inc', 'months_since_earliest_cr_line', 'total_acc', 'open_acc', 'mths_since_last_delinq', 'grade_E', 'emp_length', 'title_Other', 'inq_last_6mths', 'mths_since_last_major_derog', 'title_Debt consolidation', 'initial_list_status_w', 'grade_F', 'months_since_next_pymnt_d', 'verification_status_Verified', 'grade_D']


In [69]:
if not os.path.exists('multicollinearity_qualified_feature_list.txt'):
    Check_Multicollinearity(X_train, numerical_feature_list)

In [70]:
with open('multicollinearity_qualified_feature_list.txt') as f:
    multicollinearity_qualified_feature_list = f.read().splitlines()

In [71]:
multicollinearity_qualified_X_train = X_train[multicollinearity_qualified_feature_list].copy()

In [72]:
X_train = multicollinearity_qualified_X_train.copy()
X_train.shape

(373028, 30)

In [73]:
X_train.head()

,months_since_last_pymnt_d,last_pymnt_amnt,funded_amnt,out_prncp,total_rec_int,total_rec_late_fee,int_rate,months_since_last_credit_pull_d,term,tot_cur_bal,revol_bal,dti,total_rev_hi_lim,revol_util,annual_inc,months_since_earliest_cr_line,total_acc,open_acc,mths_since_last_delinq,grade_E,emp_length,title_Other,inq_last_6mths,mths_since_last_major_derog,title_Debt consolidation,initial_list_status_w,grade_F,months_since_next_pymnt_d,verification_status_Verified,grade_D
456615,67.0,8940.22,15000.0,0.00,1561.02,0.0,8.90,60.0,36.0,143586.0,20699.0,17.01,35100.0,59.0,80000.0,301.0,32.0,19.0,20.0,0,7.0,0,0.0,42.0,0,1,0,59.0,0,0
451541,60.0,204.24,8000.0,5746.54,2444.06,0.0,18.25,60.0,60.0,180443.0,13245.0,23.46,41300.0,32.1,44000.0,306.0,25.0,12.0,31.0,0,10.0,1,1.0,42.0,0,0,0,60.0,1,1
394474,60.0,314.65,12150.0,9282.24,3425.24,0.0,18.92,60.0,60.0,34197.0,7172.0,31.07,9800.0,73.2,27000.0,364.0,22.0,9.0,10.0,0,3.0,0,0.0,42.0,0,0,0,59.0,0,1
110294,64.0,3553.37,10000.0,0.00,858.01,0.0,6.03,64.0,36.0,77959.0,2138.0,9.16,10100.0,21.2,33000.0,229.0,17.0,5.0,31.0,0,3.0,1,0.0,42.0,0,1,0,59.0,0,0
139343,83.0,526.53,15825.0,0.00,1301.35,0.0,12.12,60.0,36.0,187370.0,30326.0,17.94,32400.0,93.6,59000.0,499.0,31.0,7.0,31.0,0,10.0,1,0.0,42.0,0,0,0,59.0,1,0


## Step-9: Remove Outliers

In [74]:
numerical_feature_list = list(X_train.select_dtypes(include=['number']).columns)
print(len(numerical_feature_list))
print(numerical_feature_list)

30
['months_since_last_pymnt_d', 'last_pymnt_amnt', 'funded_amnt', 'out_prncp', 'total_rec_int', 'total_rec_late_fee', 'int_rate', 'months_since_last_credit_pull_d', 'term', 'tot_cur_bal', 'revol_bal', 'dti', 'total_rev_hi_lim', 'revol_util', 'annual_inc', 'months_since_earliest_cr_line', 'total_acc', 'open_acc', 'mths_since_last_delinq', 'grade_E', 'emp_length', 'title_Other', 'inq_last_6mths', 'mths_since_last_major_derog', 'title_Debt consolidation', 'initial_list_status_w', 'grade_F', 'months_since_next_pymnt_d', 'verification_status_Verified', 'grade_D']


In [75]:
Remove_Outlies(X_train, y_train, numerical_feature_list)

Feature:  months_since_last_pymnt_d
Initial Min:  60.0  Initial Median:  61.0  Initial Max:  157.0
Min Cut:  60.0  Max Cut:  132.0
Data Removed:  933 ( 0.25 %) Total Data Removed:  933 ( 0.25 %)
Low Value Removed:  0  High Value Removed:  933
Final Min:  60.0  Final Median:  61.0  Final Max:  157.0


Feature:  last_pymnt_amnt
Initial Min:  0.0  Initial Median:  545.96  Initial Max:  36170.14
Min Cut:  0.0  Max Cut:  31526.948154
Data Removed:  992 ( 0.27 %) Total Data Removed:  1925 ( 0.52 %)
Low Value Removed:  0  High Value Removed:  992
Final Min:  0.0  Final Median:  545.96  Final Max:  36170.14


Feature:  funded_amnt
Initial Min:  500.0  Initial Median:  12000.0  Initial Max:  35000.0
Min Cut:  1000.0  Max Cut:  35000.0
Data Removed:  29 ( 0.01 %) Total Data Removed:  1939 ( 0.52 %)
Low Value Removed:  29  High Value Removed:  0
Final Min:  500.0  Final Median:  12000.0  Final Max:  35000.0


Feature:  out_prncp
Initial Min:  0.0  Initial Median:  443.97  Initial Max:  32160.38
M

## Step-10: Final Feature Selection

In [76]:
print(X_train.shape)
print(y_train.shape)

(359348, 30)
(359348,)


In [77]:
# now let's create feature selected dataset 
# Rrun time ~ 18 min
if not os.path.exists('selected_feature_list.txt'):
    # define parameters for feature selection
    max_validation_round = 5 # Range: 2-10, Default 10
    Make_Feature_Selection(X_train, y_train, max_validation_round)

In [78]:
with open('selected_feature_list.txt') as f:
    selected_feature_list = f.read().splitlines()

In [79]:
feature_selected_X_train = X_train[selected_feature_list].copy()

In [80]:
X_train = feature_selected_X_train.copy()
X_train.shape

(359348, 18)

## Step-11: Build Model

In [81]:
# CatBoost Model
model_CBC = CatBoostClassifier(verbose=False)

In [82]:
# Randomized Search CV param_distributions for CatBoost

# iterations: int -> 100-1000 
iterations = [int(x) for x in np.linspace(100, 1000, num = 5)]
# learning_rate: float -> 0.01–0.30
learning_rate = [round(x, 2) for x in np.linspace(0.01, 0.30, num = 5)]
# depth: int -> 2–10 
depth = [int(x) for x in np.linspace(2, 10, num = 5)]
# l2_leaf_reg: int -> 2–30 
l2_leaf_reg = [int(x) for x in np.linspace(2, 30, num = 5)]
# border_count: int -> 10-100
border_count = [int(x) for x in np.linspace(10, 100, num = 5)]


In [83]:
# Create the random grid
random_param_distributions = {
                            'iterations': iterations,
                            'learning_rate': learning_rate,
                            'depth': depth,
                            'l2_leaf_reg': l2_leaf_reg,
                            'border_count': border_count
                            }
print(random_param_distributions)

{'iterations': [100, 325, 550, 775, 1000], 'learning_rate': [0.01, 0.08, 0.16, 0.23, 0.3], 'depth': [2, 4, 6, 8, 10], 'l2_leaf_reg': [2, 9, 16, 23, 30], 'border_count': [10, 32, 55, 77, 100]}


In [84]:
# max_cross_validation
max_cross_validation = 2 # Test Value: 2, Default Value 10

In [85]:
# RandomizedSearchCV
randomized_search_cv = RandomizedSearchCV(
                                        estimator = model_CBC,
                                        param_distributions = random_param_distributions,
                                        n_iter = 20,
                                        cv = max_cross_validation, 
                                        random_state = 0,
                                        verbose = 1, 
                                        n_jobs = 1
                                        )

In [86]:
if not os.path.exists('catboost_classification_model.pkl'):
    # Run time: 17 min
    randomized_search_cv.fit(X_train, y_train)
    # Pickle the model
    with open('catboost_classification_model.pkl', 'wb') as file:
        pickle.dump(randomized_search_cv, file)

In [87]:
# load the pickle files
randomized_search_cv = pickle.load(open('catboost_classification_model.pkl','rb'))

In [88]:
model_best_score = randomized_search_cv.best_score_
print('Best score of the model: ', model_best_score)

Best score of the model:  0.9856323118536905


In [89]:
randomized_search_cv.best_params_

{'learning_rate': 0.08,
 'l2_leaf_reg': 2,
 'iterations': 775,
 'depth': 8,
 'border_count': 55}

## Step-12: Model Evaluation

### Model performance metrics for training dataset

In [90]:
y_hat = randomized_search_cv.predict(X_train)

In [91]:
# construct model performance evaluation dataframe
performance_train_dataset = X_train.copy()
performance_train_dataset['y_train'] = y_train
performance_train_dataset['y_hat'] = y_hat

In [92]:
performance_train_dataset.head()

,revol_bal,months_since_earliest_cr_line,months_since_last_credit_pull_d,last_pymnt_amnt,funded_amnt,tot_cur_bal,total_rev_hi_lim,open_acc,total_rec_late_fee,total_acc,term,total_rec_int,int_rate,out_prncp,dti,revol_util,months_since_last_pymnt_d,annual_inc,y_train,y_hat
456615,20699.0,301.0,60.0,8940.22,15000.0,143586.0,35100.0,19.0,0.0,32.0,36.0,1561.02,8.90,0.00,17.01,59.0,67.0,80000.0,1,1
451541,13245.0,306.0,60.0,204.24,8000.0,180443.0,41300.0,12.0,0.0,25.0,60.0,2444.06,18.25,5746.54,23.46,32.1,60.0,44000.0,1,1
394474,7172.0,364.0,60.0,314.65,12150.0,34197.0,9800.0,9.0,0.0,22.0,60.0,3425.24,18.92,9282.24,31.07,73.2,60.0,27000.0,1,1
110294,2138.0,229.0,64.0,3553.37,10000.0,77959.0,10100.0,5.0,0.0,17.0,36.0,858.01,6.03,0.00,9.16,21.2,64.0,33000.0,1,1
139343,30326.0,499.0,60.0,526.53,15825.0,187370.0,32400.0,7.0,0.0,31.0,36.0,1301.35,12.12,0.00,17.94,93.6,83.0,59000.0,0,0


In [93]:
confusion_matrix_train = confusion_matrix(y_train, y_hat)
print(confusion_matrix_train)

[[ 36521   2149]
 [  1117 319561]]


In [94]:
train_accuracy_score = accuracy_score(y_train, y_hat)
train_precision_score = precision_score(y_train, y_hat)
train_recall_score = recall_score(y_train, y_hat)
train_f1_score = f1_score(y_train, y_hat)

print('Train accuracy score: {:.3f}'.format(train_accuracy_score))
print('Train precision score: {:.3f}'.format(train_precision_score))
print('Train recall score: {:.3f}'.format(train_recall_score))
print('Train F1 score: {:.3f}'.format(train_f1_score))

Train accuracy score: 0.991
Train precision score: 0.993
Train recall score: 0.997
Train F1 score: 0.995


### Model performance metrics for test dataset

In [95]:
# Before Testing
# X_test and X_train should be of same data format
# dealing with missing numerical features in X_test using transform method
# keep only thore columns in X_test as that of X_train

In [96]:
print(X_train.shape)
print(X_test.shape)

(359348, 18)
(93257, 49)


In [97]:
# only keep the columns same as X_train in X_test
X_train_feature_list = X_train.columns.values.tolist()
X_test = X_test[X_train_feature_list]

In [98]:
print(X_train.shape)
print(X_test.shape)

(359348, 18)
(93257, 18)


In [99]:
X_test_feature_list = list(X_test.select_dtypes(include=['number']).columns)
print(len(X_test_feature_list))
print(X_test_feature_list)

18
['revol_bal', 'months_since_earliest_cr_line', 'months_since_last_credit_pull_d', 'last_pymnt_amnt', 'funded_amnt', 'tot_cur_bal', 'total_rev_hi_lim', 'open_acc', 'total_rec_late_fee', 'total_acc', 'term', 'total_rec_int', 'int_rate', 'out_prncp', 'dti', 'revol_util', 'months_since_last_pymnt_d', 'annual_inc']


In [100]:
# now we will rplace the missing values of numeical features with the median value of the feature as in train dataset
Replace_Missing_Numerical_Values(X_test, X_test_feature_list)
# X_test[X_test_feature_list] = imputer_median.transform(X_test[X_test_feature_list])

In [101]:
X_test.head()

,revol_bal,months_since_earliest_cr_line,months_since_last_credit_pull_d,last_pymnt_amnt,funded_amnt,tot_cur_bal,total_rev_hi_lim,open_acc,total_rec_late_fee,total_acc,term,total_rec_int,int_rate,out_prncp,dti,revol_util,months_since_last_pymnt_d,annual_inc
395346,1203.0,352.0,60.0,1159.57,1800.0,107437.0,2450.0,7.0,0.0,32.0,36.0,290.92,14.64,0.00,19.11,49.0,64.0,50000.0
376583,1921.0,270.0,60.0,185.60,6000.0,4413.0,27000.0,9.0,0.0,24.0,36.0,518.97,7.12,2992.57,7.98,7.1,60.0,63000.0
297790,3174.0,245.0,60.0,8835.32,10700.0,68042.0,5400.0,8.0,0.0,39.0,36.0,1102.28,14.99,0.00,23.35,58.8,66.0,82000.0
47347,14920.0,335.0,71.0,10563.32,15000.0,48874.0,57100.0,10.0,0.0,17.0,36.0,1343.59,9.67,0.00,28.34,26.1,72.0,60000.0
446772,26064.0,362.0,72.0,357.42,14000.0,63601.0,32500.0,11.0,0.0,35.0,60.0,1243.86,18.25,0.00,31.02,80.2,77.0,86000.0


In [102]:
Check_Missing_Values(X_test)

Following featues have more than 80% missing values:  0
[]


,Missing_Data,Missing_Data (%),Data_Type
revol_bal,0,0.0,float64
months_since_earliest_cr_line,0,0.0,float64
months_since_last_credit_pull_d,0,0.0,float64
last_pymnt_amnt,0,0.0,float64
funded_amnt,0,0.0,float64
tot_cur_bal,0,0.0,float64
total_rev_hi_lim,0,0.0,float64
open_acc,0,0.0,float64
total_rec_late_fee,0,0.0,float64
total_acc,0,0.0,float64


In [103]:
y_pred = randomized_search_cv.predict(X_test)

In [104]:
# construct model performance evaluation dataframe
performance_test_dataset = backup_X_test.copy()
performance_test_dataset['y_test'] = y_test
performance_test_dataset['y_pred'] = y_pred

In [105]:
performance_test_dataset.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,last_pymnt_amnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,months_since_issue_d,months_since_earliest_cr_line,months_since_last_pymnt_d,months_since_next_pymnt_d,months_since_last_credit_pull_d,y_test,y_pred
395346,1800,1800,1800.0,36,14.64,62.09,C,C3,biller,10.0,OWN,50000.0,Source Verified,n,home_improvement,Home improvement,328xx,FL,19.11,0.0,4.0,NaN,7.0,1.0,1203,49.0,32.0,w,0.00,0.00,2090.92,2090.92,1800.00,290.92,0.0,1159.57,0.0,31.0,1,INDIVIDUAL,0.0,965.0,107437.0,2450.0,80.0,352.0,64.0,NaN,60.0,1,1
376583,6000,6000,6000.0,36,7.12,185.60,A,A3,Implementations Project Manager,1.0,RENT,63000.0,Not Verified,n,debt_consolidation,Debt consolidation,300xx,GA,7.98,0.0,0.0,NaN,9.0,0.0,1921,7.1,24.0,w,2992.57,2992.57,3526.40,3526.40,3007.43,518.97,0.0,185.60,0.0,NaN,1,INDIVIDUAL,0.0,0.0,4413.0,27000.0,79.0,270.0,60.0,59.0,60.0,1,1
297790,10700,10700,10650.0,36,14.99,370.87,C,C5,Senior Counsel,0.0,RENT,82000.0,Source Verified,n,debt_consolidation,Debt consolidation,852xx,AZ,23.35,1.0,2.0,2.0,8.0,0.0,3174,58.8,39.0,f,0.00,0.00,11802.28,11747.13,10700.00,1102.28,0.0,8835.32,0.0,2.0,1,INDIVIDUAL,1.0,0.0,68042.0,5400.0,75.0,245.0,66.0,NaN,60.0,1,1
47347,15000,15000,15000.0,36,9.67,481.69,B,B1,toxicology technician,2.0,RENT,60000.0,Verified,n,debt_consolidation,Personal loan,342xx,FL,28.34,0.0,1.0,NaN,10.0,0.0,14920,26.1,17.0,f,0.00,0.00,16343.56,16343.56,14999.97,1343.59,0.0,10563.32,0.0,NaN,1,INDIVIDUAL,0.0,0.0,48874.0,57100.0,85.0,335.0,72.0,NaN,71.0,1,1
446772,14000,14000,13925.0,60,18.25,357.42,D,D3,maitenance work controller,10.0,MORTGAGE,86000.0,Source Verified,n,debt_consolidation,Debt consolidation,788xx,TX,31.02,0.0,0.0,NaN,11.0,0.0,26064,80.2,35.0,f,0.00,0.00,4004.60,3983.15,900.66,1243.86,0.0,357.42,0.0,NaN,1,INDIVIDUAL,0.0,0.0,63601.0,32500.0,83.0,362.0,77.0,NaN,72.0,0,0


In [106]:
test_confusion_matrix = confusion_matrix(y_test, y_pred)
print(test_confusion_matrix)

[[ 9568   626]
 [ 1753 81310]]


In [107]:
test_accuracy_score = accuracy_score(y_test, y_pred)
test_precision_score = precision_score(y_test, y_pred)
test_recall_score = recall_score(y_test, y_pred)
test_f1_score = f1_score(y_test, y_pred)

print('Test accuracy score: {:.3f}'.format(test_accuracy_score))
print('Test precision score: {:.3f}'.format(test_precision_score))
print('Test recall score: {:.3f}'.format(test_recall_score))
print('Test F1 score: {:.3f}'.format(test_f1_score))

Test accuracy score: 0.974
Test precision score: 0.992
Test recall score: 0.979
Test F1 score: 0.986


### Model evaluation summary

In [108]:
# difference between nrmse_train and nrmse_test
f1_difference = abs(test_f1_score - train_f1_score)*100/train_f1_score

print('Train accuracy score: {:.3f}'.format(train_accuracy_score))
print('Test accuracy score: {:.3f}'.format(test_accuracy_score))

print('Train F1 score: {:.3f}'.format(train_f1_score))
print('Test F1 score: {:.3f}'.format(test_f1_score))

print('Difference between train_f1_score and test_f1_score: {:.2f} %'.format(f1_difference))

Train accuracy score: 0.991
Test accuracy score: 0.974
Train F1 score: 0.995
Test F1 score: 0.986
Difference between train_f1_score and test_f1_score: 0.94 %
